In [1]:
pip install pyspark

     |████████████████████████████████| 212.3MB 70kB/s 
     |████████████████████████████████| 204kB 36.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=5d44940766b706c20e7cf6f40b90c68c4fb48cb3594a20361c600759e9a957f5
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [2]:
from pyspark import SparkConf, SparkContext
import collections
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ML').getOrCreate()

In [4]:
df = spark.read.csv('/content/drive/MyDrive/XLDLL/iris.data',header=True, inferSchema= True)
#df.printSchema()

In [5]:
k = len(df.columns)
for i in range(k):
    old_name = df.columns[i]
    new_name = 'f' + str(i)
    print(old_name,new_name)
    df = df.withColumnRenamed(old_name,new_name)

5.1 f0
3.5 f1
1.4 f2
0.2 f3
Iris-setosa f4


In [6]:
df.show()

+---+---+---+---+-----------+
| f0| f1| f2| f3|         f4|
+---+---+---+---+-----------+
|4.9|3.0|1.4|0.2|Iris-setosa|
|4.7|3.2|1.3|0.2|Iris-setosa|
|4.6|3.1|1.5|0.2|Iris-setosa|
|5.0|3.6|1.4|0.2|Iris-setosa|
|5.4|3.9|1.7|0.4|Iris-setosa|
|4.6|3.4|1.4|0.3|Iris-setosa|
|5.0|3.4|1.5|0.2|Iris-setosa|
|4.4|2.9|1.4|0.2|Iris-setosa|
|4.9|3.1|1.5|0.1|Iris-setosa|
|5.4|3.7|1.5|0.2|Iris-setosa|
|4.8|3.4|1.6|0.2|Iris-setosa|
|4.8|3.0|1.4|0.1|Iris-setosa|
|4.3|3.0|1.1|0.1|Iris-setosa|
|5.8|4.0|1.2|0.2|Iris-setosa|
|5.7|4.4|1.5|0.4|Iris-setosa|
|5.4|3.9|1.3|0.4|Iris-setosa|
|5.1|3.5|1.4|0.3|Iris-setosa|
|5.7|3.8|1.7|0.3|Iris-setosa|
|5.1|3.8|1.5|0.3|Iris-setosa|
|5.4|3.4|1.7|0.2|Iris-setosa|
+---+---+---+---+-----------+
only showing top 20 rows



In [7]:
import pandas as pd
pd.DataFrame(df.take(5),columns = df.columns).transpose()

,0,1,2,3,4
f0,4.9,4.7,4.6,5,5.4
f1,3,3.2,3.1,3.6,3.9
f2,1.4,1.3,1.5,1.4,1.7
f3,0.2,0.2,0.2,0.2,0.4
f4,Iris-setosa,Iris-setosa,Iris-setosa,Iris-setosa,Iris-setosa


In [9]:
class_name = df.columns[len(df.columns)-1]
df.groupby(class_name).count().toPandas()

,f4,count
0,Iris-virginica,50
1,Iris-setosa,49
2,Iris-versicolor,50


In [10]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()
indexer.setInputCol(class_name).setOutputCol('label')
dfl = indexer.fit(df).transform(df)

In [11]:
dfl.show()

+---+---+---+---+-----------+-----+
| f0| f1| f2| f3|         f4|label|
+---+---+---+---+-----------+-----+
|4.9|3.0|1.4|0.2|Iris-setosa|  2.0|
|4.7|3.2|1.3|0.2|Iris-setosa|  2.0|
|4.6|3.1|1.5|0.2|Iris-setosa|  2.0|
|5.0|3.6|1.4|0.2|Iris-setosa|  2.0|
|5.4|3.9|1.7|0.4|Iris-setosa|  2.0|
|4.6|3.4|1.4|0.3|Iris-setosa|  2.0|
|5.0|3.4|1.5|0.2|Iris-setosa|  2.0|
|4.4|2.9|1.4|0.2|Iris-setosa|  2.0|
|4.9|3.1|1.5|0.1|Iris-setosa|  2.0|
|5.4|3.7|1.5|0.2|Iris-setosa|  2.0|
|4.8|3.4|1.6|0.2|Iris-setosa|  2.0|
|4.8|3.0|1.4|0.1|Iris-setosa|  2.0|
|4.3|3.0|1.1|0.1|Iris-setosa|  2.0|
|5.8|4.0|1.2|0.2|Iris-setosa|  2.0|
|5.7|4.4|1.5|0.4|Iris-setosa|  2.0|
|5.4|3.9|1.3|0.4|Iris-setosa|  2.0|
|5.1|3.5|1.4|0.3|Iris-setosa|  2.0|
|5.7|3.8|1.7|0.3|Iris-setosa|  2.0|
|5.1|3.8|1.5|0.3|Iris-setosa|  2.0|
|5.4|3.4|1.7|0.2|Iris-setosa|  2.0|
+---+---+---+---+-----------+-----+
only showing top 20 rows



In [15]:
class_name = 'label'
feature_names = df.columns[:-1]
print(class_name)
print(feature_names)

label
['f0', 'f1', 'f2', 'f3']


In [16]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler()
assembler.setInputCols(feature_names).setOutputCol('data')
transformed_data = assembler.transform(dfl)

In [17]:
transformed_data.show()

+---+---+---+---+-----------+-----+-----------------+
| f0| f1| f2| f3|         f4|label|             data|
+---+---+---+---+-----------+-----+-----------------+
|4.9|3.0|1.4|0.2|Iris-setosa|  2.0|[4.9,3.0,1.4,0.2]|
|4.7|3.2|1.3|0.2|Iris-setosa|  2.0|[4.7,3.2,1.3,0.2]|
|4.6|3.1|1.5|0.2|Iris-setosa|  2.0|[4.6,3.1,1.5,0.2]|
|5.0|3.6|1.4|0.2|Iris-setosa|  2.0|[5.0,3.6,1.4,0.2]|
|5.4|3.9|1.7|0.4|Iris-setosa|  2.0|[5.4,3.9,1.7,0.4]|
|4.6|3.4|1.4|0.3|Iris-setosa|  2.0|[4.6,3.4,1.4,0.3]|
|5.0|3.4|1.5|0.2|Iris-setosa|  2.0|[5.0,3.4,1.5,0.2]|
|4.4|2.9|1.4|0.2|Iris-setosa|  2.0|[4.4,2.9,1.4,0.2]|
|4.9|3.1|1.5|0.1|Iris-setosa|  2.0|[4.9,3.1,1.5,0.1]|
|5.4|3.7|1.5|0.2|Iris-setosa|  2.0|[5.4,3.7,1.5,0.2]|
|4.8|3.4|1.6|0.2|Iris-setosa|  2.0|[4.8,3.4,1.6,0.2]|
|4.8|3.0|1.4|0.1|Iris-setosa|  2.0|[4.8,3.0,1.4,0.1]|
|4.3|3.0|1.1|0.1|Iris-setosa|  2.0|[4.3,3.0,1.1,0.1]|
|5.8|4.0|1.2|0.2|Iris-setosa|  2.0|[5.8,4.0,1.2,0.2]|
|5.7|4.4|1.5|0.4|Iris-setosa|  2.0|[5.7,4.4,1.5,0.4]|
|5.4|3.9|1.3|0.4|Iris-setosa

In [34]:
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType
from scipy.spatial import distance

fixed_entry = [5.1,3.5,1.4,0.2] #for example, the entry against which you want distances
distance_udf = F.udf(lambda x: float(distance.euclidean(x, fixed_entry)), FloatType())
transformed_data = transformed_data.withColumn('distances', distance_udf(F.col('data')))

In [35]:
transformed_data.show()

+---+---+---+---+-----------+-----+-----------------+----------+
| f0| f1| f2| f3|         f4|label|             data| distances|
+---+---+---+---+-----------+-----+-----------------+----------+
|4.9|3.0|1.4|0.2|Iris-setosa|  2.0|[4.9,3.0,1.4,0.2]|0.53851646|
|4.7|3.2|1.3|0.2|Iris-setosa|  2.0|[4.7,3.2,1.3,0.2]|0.50990194|
|4.6|3.1|1.5|0.2|Iris-setosa|  2.0|[4.6,3.1,1.5,0.2]| 0.6480741|
|5.0|3.6|1.4|0.2|Iris-setosa|  2.0|[5.0,3.6,1.4,0.2]|0.14142136|
|5.4|3.9|1.7|0.4|Iris-setosa|  2.0|[5.4,3.9,1.7,0.4]| 0.6164414|
|4.6|3.4|1.4|0.3|Iris-setosa|  2.0|[4.6,3.4,1.4,0.3]|0.51961523|
|5.0|3.4|1.5|0.2|Iris-setosa|  2.0|[5.0,3.4,1.5,0.2]|0.17320508|
|4.4|2.9|1.4|0.2|Iris-setosa|  2.0|[4.4,2.9,1.4,0.2]|0.92195445|
|4.9|3.1|1.5|0.1|Iris-setosa|  2.0|[4.9,3.1,1.5,0.1]| 0.4690416|
|5.4|3.7|1.5|0.2|Iris-setosa|  2.0|[5.4,3.7,1.5,0.2]|0.37416574|
|4.8|3.4|1.6|0.2|Iris-setosa|  2.0|[4.8,3.4,1.6,0.2]|0.37416574|
|4.8|3.0|1.4|0.1|Iris-setosa|  2.0|[4.8,3.0,1.4,0.1]|  0.591608|
|4.3|3.0|1.1|0.1|Iris-set

In [36]:
transformed_data = transformed_data.sort(transformed_data.distances.asc())
transformed_data.filter(transformed_data.distances > 0).take(3)

[Row(f0=5.1, f1=3.5, f2=1.4, f3=0.3, f4='Iris-setosa', label=2.0, data=DenseVector([5.1, 3.5, 1.4, 0.3]), distances=0.10000000149011612),
 Row(f0=5.1, f1=3.4, f2=1.5, f3=0.2, f4='Iris-setosa', label=2.0, data=DenseVector([5.1, 3.4, 1.5, 0.2]), distances=0.1414213627576828),
 Row(f0=5.2, f1=3.4, f2=1.4, f3=0.2, f4='Iris-setosa', label=2.0, data=DenseVector([5.2, 3.4, 1.4, 0.2]), distances=0.1414213627576828)]